# SPARK DATAFRAME API

### 1. SPARK 환경 설정 & spark 세션 연결

In [ ]:
!pip3 install pyspark

In [37]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import types as T
from pyspark.sql import window as W
from pyspark.sql import functions as F

spark = SparkSession.builder \
        .master("local") \
        .appName("Colab") \
        .getOrCreate()

### 2. 데이터 읽기

In [ ]:
## 데이터 읽기

df = spark.read.option("header", "true").csv('/content/test1.csv')

df = spark.read.csv('/content/test1.csv', header=True)

In [ ]:
df.show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Paul| 31|        10| 30000|
|  Tom| 30|         8| 25000|
|Sunny| 29|         4| 20000|
| Paul| 24|         3| 20000|
|  Ann| 21|         1| 15000|
|Peter| 23|         2| 18000|
+-----+---+----------+------+



In [ ]:
# schema 확인

df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [ ]:
df.dtypes

[('Name', 'string'),
 ('age', 'string'),
 ('Experience', 'string'),
 ('Salary', 'string')]

In [ ]:
# 타입 확인

type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
# 다른 방법으로 보기

# df.show(2)

df.show(1, vertical=True, truncate=False)

-RECORD 0-----------
 Name       | Paul  
 age        | 31    
 Experience | 10    
 Salary     | 30000 
only showing top 1 row



In [ ]:
# 일부 row만 보기

df.limit(3).show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Paul| 31|        10| 30000|
|  Tom| 30|         8| 25000|
|Sunny| 29|         4| 20000|
+-----+---+----------+------+



In [41]:
# 컬럼 확인

df.columns

['Name', 'age', 'Experience', 'Salary']

In [ ]:
# row count 확인

df.count()

6

In [ ]:
# Row 형태로 보기

df.collect()

[Row(Name='Paul', age='31', Experience='10', Salary='30000'),
 Row(Name='Tom', age='30', Experience='8', Salary='25000'),
 Row(Name='Sunny', age='29', Experience='4', Salary='20000'),
 Row(Name='Paul', age='24', Experience='3', Salary='20000'),
 Row(Name='Ann', age='21', Experience='1', Salary='15000'),
 Row(Name='Peter', age='23', Experience='2', Salary='18000')]

In [ ]:
df.first()

Row(Name='Paul', age='31', Experience='10', Salary='30000')

In [ ]:
first_row = df.first()

first_row.Name, first_row.age, first_row.Experience

('Paul', '31', '10')

### 3. 컬럼 다루기 & Dataframe 생성

In [ ]:
# 컬럼 선택
df.columns

df.select('Name', 'age').show()

df.select(['Name', 'age']).show()

+-----+---+
| Name|age|
+-----+---+
| Paul| 31|
|  Tom| 30|
|Sunny| 29|
| Paul| 24|
|  Ann| 21|
|Peter| 23|
+-----+---+



In [ ]:
# 컬럼 삭제
df_drop = df.drop('Experience')
df_drop.show()

+-----+---+------+
| Name|age|Salary|
+-----+---+------+
| Paul| 31| 30000|
|  Tom| 30| 25000|
|Sunny| 29| 20000|
| Paul| 24| 20000|
|  Ann| 21| 15000|
|Peter| 23| 18000|
+-----+---+------+



In [40]:
# 컬럼 추가 - withColumn

df.withColumn('age1', F.col('Name')).show()

+-----+---+----------+------+----+
| Name|age|Experience|Salary|age1|
+-----+---+----------+------+----+
| Paul| 31|        10| 30000|NULL|
|  Tom| 30|         8| 25000|NULL|
|Sunny| 29|         4| 20000|NULL|
| Paul| 24|         3| 20000|NULL|
|  Ann| 21|         1| 15000|NULL|
|Peter| 23|         2| 18000|NULL|
+-----+---+----------+------+----+



In [ ]:
# 컬럼 추가 - expr

df.select("*", F.expr("age * 2 as age2"))

In [ ]:
# 컬럼 추가 - selectExpr

df.selectExpr("Name", "age", "Experience", "age * 2 as age2")

In [ ]:
# 컬럼 이름 변경
# df.withColumnRenamed('기존 컬럼 이름', '변경될 컬럼 이름')

df.withColumnRenamed('age', 'new_age').show()

+-----+-------+----------+------+
| Name|new_age|Experience|Salary|
+-----+-------+----------+------+
| Paul|     31|        10| 30000|
|  Tom|     30|         8| 25000|
|Sunny|     29|         4| 20000|
| Paul|     24|         3| 20000|
|  Ann|     21|         1| 15000|
|Peter|     23|         2| 18000|
+-----+-------+----------+------+



In [ ]:
# 컬럼 타입 변경
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [42]:
df.withColumn('age', F.col('age').cast(T.IntegerType())).printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [46]:
# 새로운 dataframe 생성

schema = T.StructType(
    [
        T.StructField('aaa', T.StringType(), True),
        T.StructField('bbb', T.StringType(), True),
        T.StructField('ccc', T.StringType(), True),
    ]
)

data_1 = [
    Row('apple', 'banana', 'tomato'),
    Row('apple1', 'banana1', 'tomato1'),
    Row('apple2', 'banana2', 'tomato2'),
]

data_2 = [
    ('apple', 'banana', 'tomato'),
    ('apple3', 'banana3', 'tomato3'),
    ('apple4', 'banana4', 'tomato4')
]

df_new = spark.createDataFrame(data_2, schema)

df_new.show()

+------+-------+-------+
|   aaa|    bbb|    ccc|
+------+-------+-------+
| apple| banana| tomato|
|apple3|banana3|tomato3|
|apple4|banana4|tomato4|
+------+-------+-------+



### 4. Dataframe 쓰기 및 옵션들

In [ ]:
# dataframe 쓰기 -> write

df.write.mode('overwrite').csv("저장될 경로")

df.coalesce(1).write.mode('overwrite').csv("저장될 경로")

##### 1) read 옵션
  - PERMISSIVE(default) : 타입 이상이 있으면 null로 처리 후 read
  - DROPMALFORMED : 타입 이상 있는 row drop
  - FAILFAST : 타입 이상 있는 row 발견시 실행 실패

In [ ]:
_schema = "employee_id int, department_id int, name string, age int, gender string, salary double, hire_date date, _corrupt_record string"

emp = spark.read.schema(_schema).csv("/content/emp.csv", header=True, inferSchema=True)

emp = spark.read.schema(_schema).option("mode", "PERMISSIVE").csv("/content/emp.csv", header=True, inferSchema=True)

In [ ]:
# options로 옵션 부여

_options = {
    "header" : "true",
    "inferSchema" : "true",
    "mode" : "PERMISSIVE", # DROPMALFORMED, FAILFAST
    "columnNameOfCorruptRecord" : "bad_record"
}

_schema = "employee_id int, department_id int, name string, age int, gender string, salary double, hire_date date, bad_record string"

emp = spark.read.schema(_schema).options(**_options).csv("/content/emp_new.csv", header=True, inferSchema=True)

emp.show()

##### 2) write 옵션
  - error(default) : 파일 있으면 실행 실패
  - ignore : 파일 있으면 작업 안함
  - overwrite : 기존 경로 덮어쓰기
  - append : 기존 경로에 추가

In [ ]:
df.write.mode('overwrite').csv("저장될 경로")

### 5. 결측치 다루기

In [48]:
df = spark.read.csv('/content/test2.csv', header=True)
df.show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Lucy|NULL|      NULL| 40000|
| NULL|  34|        10| 38000|
| NULL|  36|      NULL|  NULL|
+-----+----+----------+------+



In [50]:
# 결측치 drop 하기

df.na.drop().show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Paul| 31|        10| 30000|
| Lucy| 30|         8| 25000|
|Sunny| 29|         4| 20000|
| Paul| 24|         3| 20000|
|  Ann| 21|         1| 15000|
|  Tom| 23|         2| 18000|
+-----+---+----------+------+



In [51]:
df.dropna().show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Paul| 31|        10| 30000|
| Lucy| 30|         8| 25000|
|Sunny| 29|         4| 20000|
| Paul| 24|         3| 20000|
|  Ann| 21|         1| 15000|
|  Tom| 23|         2| 18000|
+-----+---+----------+------+



In [56]:
# thresh : 정상값이 thresh개 보다 적게 있는 row를 drop

df.dropna(thresh=2).show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Lucy|NULL|      NULL| 40000|
| NULL|  34|        10| 38000|
+-----+----+----------+------+



In [58]:
# any : null값이 하나라도 있으면 제거
# all : 모든 row가 null인 row만 제거

df.na.drop(how='all').show()


+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Lucy|NULL|      NULL| 40000|
| NULL|  34|        10| 38000|
| NULL|  36|      NULL|  NULL|
+-----+----+----------+------+



In [62]:
df_ = spark.read.csv('/content/test22.csv', header=True)
df_.na.drop(how='all').show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Paul| 31|        10| 30000|
| Lucy| 30|         8| 25000|
|Sunny| 29|         4| 20000|
| Paul| 24|         3| 20000|
|  Ann| 21|         1| 15000|
|  Tom| 23|         2| 18000|
| NULL| 34|        10| 38000|
+-----+---+----------+------+



In [63]:
df.show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Lucy|NULL|      NULL| 40000|
| NULL|  34|        10| 38000|
| NULL|  36|      NULL|  NULL|
+-----+----+----------+------+



In [64]:
# 특정 컬럼에 대해서

df.na.drop(how='all', subset = ['age', 'Experience']).show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Paul| 31|        10| 30000|
| Lucy| 30|         8| 25000|
|Sunny| 29|         4| 20000|
| Paul| 24|         3| 20000|
|  Ann| 21|         1| 15000|
|  Tom| 23|         2| 18000|
| NULL| 34|        10| 38000|
| NULL| 36|      NULL|  NULL|
+-----+---+----------+------+



In [68]:
# 결측치 채우기, value가 해당 컬럼의 type과 맞아야 한다.

df.na.fill(value = '가나다', subset = ['Name', 'Salary']).show()

+------+----+----------+------+
|  Name| age|Experience|Salary|
+------+----+----------+------+
|  Paul|  31|        10| 30000|
|  Lucy|  30|         8| 25000|
| Sunny|  29|         4| 20000|
|  Paul|  24|         3| 20000|
|   Ann|  21|         1| 15000|
|   Tom|  23|         2| 18000|
|  Lucy|NULL|      NULL| 40000|
|가나다|  34|        10| 38000|
|가나다|  36|      NULL|가나다|
+------+----+----------+------+



In [69]:
df.fillna(value = '컴퓨터', subset = ['age', 'Experience', 'Salary']).show()

+-----+------+----------+------+
| Name|   age|Experience|Salary|
+-----+------+----------+------+
| Paul|    31|        10| 30000|
| Lucy|    30|         8| 25000|
|Sunny|    29|         4| 20000|
| Paul|    24|         3| 20000|
|  Ann|    21|         1| 15000|
|  Tom|    23|         2| 18000|
| Lucy|컴퓨터|    컴퓨터| 40000|
| NULL|    34|        10| 38000|
| NULL|    36|    컴퓨터|컴퓨터|
+-----+------+----------+------+



### 6. filter

In [73]:
item = spark.read.csv("/content/item_his.csv", header=True)
item.show(20)

+-----+-----+-------+--------+----------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|  codename|   mascodename|payprice|price|
+-----+-----+-------+--------+----------+--------------+--------+-----+
|53687|175.0| 202305|20230508|  액세서리|아바타파츠구분|      50|   50|
|53687|175.0| 202305|20230508|상태메시지|  기타파츠구분|      50|   50|
|20163|161.0| 202304|20230408|  액세서리|아바타파츠구분|      50|   50|
|88572| 14.0| 202304|20230408|상태메시지|  기타파츠구분|      50|   50|
|88572| 14.0| 202304|20230408|상태메시지|  기타파츠구분|      50|   50|
|26112|130.0| 202304|20230405|  액세서리|아바타파츠구분|      50|   50|
|26112|130.0| 202304|20230405|      헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|      얼굴|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|      신발|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|상태메시지|  기타파츠구분|      50|   50|
|49541|170.0| 202306|20230630|      헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|      헤어|아바타파츠구분|     150|  150|
|37822| 39.0| 202306|20230608|      헤어|아바타

In [77]:
# 단일 필터

item.filter(F.col('codename') == '액세서리').show(5)

+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|53687|175.0| 202305|20230508|액세서리|아바타파츠구분|      50|   50|
|20163|161.0| 202304|20230408|액세서리|아바타파츠구분|      50|   50|
|26112|130.0| 202304|20230405|액세서리|아바타파츠구분|      50|   50|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
+-----+-----+-------+--------+--------+--------------+--------+-----+
only showing top 5 rows



In [79]:
item = item.withColumn('payprice', F.col('payprice').cast(T.IntegerType()))

item.printSchema()

root
 |-- idx: string (nullable = true)
 |-- lv: string (nullable = true)
 |-- proc_ym: string (nullable = true)
 |-- proc_ymd: string (nullable = true)
 |-- codename: string (nullable = true)
 |-- mascodename: string (nullable = true)
 |-- payprice: integer (nullable = true)
 |-- price: string (nullable = true)



In [82]:
item.filter(F.col('payprice') < 50).show(5)

+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|32960|151.0| 202305|20230510|액세서리|아바타파츠구분|      25|   25|
|61823|151.0| 202304|20230418|액세서리|아바타파츠구분|      25|   25|
|77389|118.0| 202304|20230421|액세서리|아바타파츠구분|      25|   25|
+-----+-----+-------+--------+--------+--------------+--------+-----+
only showing top 5 rows



In [85]:
# 다중 필터

item.filter(F.col('codename') == '액세서리').filter(F.col('payprice') < 50).show(5)

+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|32960|151.0| 202305|20230510|액세서리|아바타파츠구분|      25|   25|
|61823|151.0| 202304|20230418|액세서리|아바타파츠구분|      25|   25|
|77389|118.0| 202304|20230421|액세서리|아바타파츠구분|      25|   25|
+-----+-----+-------+--------+--------+--------------+--------+-----+
only showing top 5 rows



In [88]:
# df.filter((조건1) & (조건2) | (조건3) .....)

item.filter((F.col('codename') == '액세서리') | (F.col('payprice') < 50)).show(5)

+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|53687|175.0| 202305|20230508|액세서리|아바타파츠구분|      50|   50|
|20163|161.0| 202304|20230408|액세서리|아바타파츠구분|      50|   50|
|26112|130.0| 202304|20230405|액세서리|아바타파츠구분|      50|   50|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
+-----+-----+-------+--------+--------+--------------+--------+-----+
only showing top 5 rows



In [91]:
# 필터 반대 조건

# item.filter(F.col('codename') != '액세서리').show()

item.filter(~(F.col('codename') != '액세서리')).show(5)

+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|53687|175.0| 202305|20230508|액세서리|아바타파츠구분|      50|   50|
|20163|161.0| 202304|20230408|액세서리|아바타파츠구분|      50|   50|
|26112|130.0| 202304|20230405|액세서리|아바타파츠구분|      50|   50|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
|76257|190.0| 202306|20230608|액세서리|아바타파츠구분|      25|   25|
+-----+-----+-------+--------+--------+--------------+--------+-----+
only showing top 5 rows



In [93]:
# 중복 제거 -> distinct, dropDuplicates, drop_duplicates

item.distinct()

83064

In [97]:
item.dropDuplicates(subset = ['codename', 'price']).count()

12

In [102]:
# 컬럼에 특정 value 포함 여부 확인 (isin)

item.filter(F.col('codename').isin(['헤어', '얼굴', '신발'])).show(5)

+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|26112|130.0| 202304|20230405|    헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    얼굴|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    신발|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    헤어|아바타파츠구분|     150|  150|
+-----+-----+-------+--------+--------+--------------+--------+-----+
only showing top 5 rows



In [105]:
# Null값 확인 -> isNull(), isNotNull()

df.filter(F.col('Name').isNull()).show()

df.filter(F.col('Name').isNotNull()).show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Lucy|NULL|      NULL| 40000|
+-----+----+----------+------+



In [111]:
# 비슷한 형태 확인 (like) -> %아바타%, 아바다_

item.filter(F.col('mascodename').like('아바타%')).show(5)

+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|53687|175.0| 202305|20230508|액세서리|아바타파츠구분|      50|   50|
|20163|161.0| 202304|20230408|액세서리|아바타파츠구분|      50|   50|
|26112|130.0| 202304|20230405|액세서리|아바타파츠구분|      50|   50|
|26112|130.0| 202304|20230405|    헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    얼굴|아바타파츠구분|     150|  150|
+-----+-----+-------+--------+--------+--------------+--------+-----+
only showing top 5 rows



In [117]:
# When 조건문

df.withColumn('new_age', F.when(F.col('age').isNull(), '-').otherwise(F.col('Salary')*2)).show()

+-----+----+----------+------+-------+
| Name| age|Experience|Salary|new_age|
+-----+----+----------+------+-------+
| Paul|  31|        10| 30000|60000.0|
| Lucy|  30|         8| 25000|50000.0|
|Sunny|  29|         4| 20000|40000.0|
| Paul|  24|         3| 20000|40000.0|
|  Ann|  21|         1| 15000|30000.0|
|  Tom|  23|         2| 18000|36000.0|
| Lucy|NULL|      NULL| 40000|      -|
| NULL|  34|        10| 38000|76000.0|
| NULL|  36|      NULL|  NULL|   NULL|
+-----+----+----------+------+-------+



In [121]:
# Between 조건문

item.filter(F.col('payprice').between(100,200)).show()

+-----+-----+-------+--------+--------+--------------+--------+-----+
|  idx|   lv|proc_ym|proc_ymd|codename|   mascodename|payprice|price|
+-----+-----+-------+--------+--------+--------------+--------+-----+
|26112|130.0| 202304|20230405|    헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    얼굴|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    신발|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    헤어|아바타파츠구분|     150|  150|
|49541|170.0| 202306|20230630|    헤어|아바타파츠구분|     150|  150|
|37822| 39.0| 202306|20230608|    헤어|아바타파츠구분|     150|  150|
|84801|112.0| 202306|20230627|    헤어|아바타파츠구분|     150|  150|
|84801|112.0| 202306|20230627|    얼굴|아바타파츠구분|     150|  150|
|67188|175.0| 202306|20230612|    얼굴|아바타파츠구분|     150|  150|
|67188|175.0| 202306|20230612|    상의|아바타파츠구분|     200|  200|
|67188|175.0| 202306|20230612|    헤어|아바타파츠구분|     150|  150|
| 3663|192.0| 202306|20230612|    얼굴|아바타파츠구분|     150|  150|
| 6533|164.0| 202305|20230511|    헤어|아바타파츠구분|     150|  15

### 7. group by + aggregation

In [126]:
# group by 함수 -> sum, count

item.groupby(F.col('codename')).count().show()

item.groupby('codename').mean().show()

+----------+-----+
|  codename|count|
+----------+-----+
|      신발|17245|
|      하의| 9198|
|  액세서리|11570|
|      얼굴|14075|
|      헤어|24011|
|    코스튬| 4697|
|상태메시지|17438|
|      상의|11311|
+----------+-----+

+----------+------------------+
|  codename|     avg(payprice)|
+----------+------------------+
|      신발| 74.39373731516382|
|      하의|             250.0|
|  액세서리| 44.81201382886776|
|      얼굴|             150.0|
|      헤어|             150.0|
|    코스튬|             300.0|
|상태메시지| 59.50510379630692|
|      상의|198.15224118115108|
+----------+------------------+



In [128]:
# group by + aggregation 함수

item.groupby('codename').agg(F.count('codename').alias('countcount')).show()

+----------+----------+
|  codename|countcount|
+----------+----------+
|      신발|     17245|
|      하의|      9198|
|  액세서리|     11570|
|      얼굴|     14075|
|      헤어|     24011|
|    코스튬|      4697|
|상태메시지|     17438|
|      상의|     11311|
+----------+----------+



In [130]:
item.groupby('codename').agg(F.mean(F.col('payprice')).alias('codename_mean')).show()

+----------+------------------+
|  codename|     codename_mean|
+----------+------------------+
|      신발| 74.39373731516382|
|      하의|             250.0|
|  액세서리| 44.81201382886776|
|      얼굴|             150.0|
|      헤어|             150.0|
|    코스튬|             300.0|
|상태메시지| 59.50510379630692|
|      상의|198.15224118115108|
+----------+------------------+



In [132]:
item.groupby('codename').agg(
    F.mean(F.col('payprice')).alias('codename_mean'),
    F.max(F.col('payprice')).alias('codename_max'),
    F.min(F.col('payprice')).alias('codename_min'),
    F.round(F.mean(F.col('payprice')), 2).alias('codename_round')
).show()

+----------+------------------+------------+------------+--------------+
|  codename|     codename_mean|codename_max|codename_min|codename_round|
+----------+------------------+------------+------------+--------------+
|      신발| 74.39373731516382|         150|          60|         74.39|
|      하의|             250.0|         250|         250|         250.0|
|  액세서리| 44.81201382886776|          50|          25|         44.81|
|      얼굴|             150.0|         150|         150|         150.0|
|      헤어|             150.0|         150|         150|         150.0|
|    코스튬|             300.0|         300|         300|         300.0|
|상태메시지| 59.50510379630692|         100|          50|         59.51|
|      상의|198.15224118115108|         200|         150|        198.15|
+----------+------------------+------------+------------+--------------+



In [135]:
item.groupby('codename').agg(
    F.collect_list(F.col('payprice')).alias('codename_list'),
    F.collect_set(F.col('payprice')).alias('codename_set')
).show()

+----------+--------------------+------------+
|  codename|       codename_list|codename_set|
+----------+--------------------+------------+
|      신발|[150, 60, 60, 60,...|   [150, 60]|
|      하의|[250, 250, 250, 2...|       [250]|
|  액세서리|[50, 50, 50, 25, ...|    [50, 25]|
|      얼굴|[150, 150, 150, 1...|       [150]|
|      헤어|[150, 150, 150, 1...|       [150]|
|    코스튬|[300, 300, 300, 3...|       [300]|
|상태메시지|[50, 50, 50, 50, ...|   [100, 50]|
|      상의|[200, 200, 200, 2...|  [150, 200]|
+----------+--------------------+------------+



In [ ]:
df_groupby = \
    df.groupby('col_a') \
        .agg(
            F.sum(F.col('col_a')), # 총합
            F.countDistinct(F.col('col_a')), # distinct한 개수만 세기
            F.count(F.col('col_a')), # 전체 개수 세기
            F.mean(F.col('col_a')), # 평균값
            F.avg(F.col('col_a')), # 평균값
            F.stddev(F.col('col_a')), # 표준편차
            F.min(F.col('')).alias('min'), # 최솟값
            F.max(F.col('')).alias('max'), # 최댓값
            F.round(F.avg('col_a'), 2), # 반올림
            F.collect_list(F.col('col_b')), # group by 후 특정 컬럼의 값들을 list로 묶어준다.(중복 O)
            F.collect_set(F.col('col_b')) # group by 후 특정 컬럼의 값들을 list로 묶어준다.(중복 X)
        )

### 8. order by

In [143]:
# df = df.withColumn('Experience', F.col('Experience').cast(T.IntegerType()))
df = df.withColumn('age', F.col('age').cast(T.IntegerType()))
df = df.withColumn('Salary', F.col('Salary').cast(T.IntegerType()))
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [145]:
# 1) 기본

df.orderBy('Salary').show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| NULL|  36|      NULL|  NULL|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
| Lucy|  30|         8| 25000|
| Paul|  31|        10| 30000|
| NULL|  34|        10| 38000|
| Lucy|NULL|      NULL| 40000|
+-----+----+----------+------+



In [146]:
# 2) 어려개

df.orderBy('age', 'Experience').show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Lucy|NULL|      NULL| 40000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Paul|  24|         3| 20000|
|Sunny|  29|         4| 20000|
| Lucy|  30|         8| 25000|
| Paul|  31|        10| 30000|
| NULL|  34|        10| 38000|
| NULL|  36|      NULL|  NULL|
+-----+----+----------+------+



In [147]:
ddf = spark.read.csv('/content/test1.csv', header=True)
ddf = ddf.withColumn('age', F.col('age').cast(T.IntegerType()))
ddf = ddf.withColumn('Experience', F.col('Experience').cast(T.IntegerType()))
ddf = ddf.withColumn('Salary', F.col('Salary').cast(T.IntegerType()))
ddf.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [148]:
ddf.orderBy('age', 'Salary').show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
|  Ann| 21|         1| 15000|
|Peter| 23|         2| 18000|
| Paul| 24|         3| 20000|
|Sunny| 29|         4| 40000|
|  Tom| 30|         8| 25000|
| Paul| 31|        10| 10000|
+-----+---+----------+------+



In [149]:
ddf.orderBy('Salary', 'age').show()

+-----+---+----------+------+
| Name|age|Experience|Salary|
+-----+---+----------+------+
| Paul| 31|        10| 10000|
|  Ann| 21|         1| 15000|
|Peter| 23|         2| 18000|
| Paul| 24|         3| 20000|
|  Tom| 30|         8| 25000|
|Sunny| 29|         4| 40000|
+-----+---+----------+------+



In [151]:
df.orderBy('age', ascending=False).show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| NULL|  36|      NULL|  NULL|
| NULL|  34|        10| 38000|
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Tom|  23|         2| 18000|
|  Ann|  21|         1| 15000|
| Lucy|NULL|      NULL| 40000|
+-----+----+----------+------+



In [152]:
df.orderBy(F.col('age').desc()).show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| NULL|  36|      NULL|  NULL|
| NULL|  34|        10| 38000|
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Tom|  23|         2| 18000|
|  Ann|  21|         1| 15000|
| Lucy|NULL|      NULL| 40000|
+-----+----+----------+------+



In [153]:
# 3) 여러개 + 정렬 순서 반대

df.orderBy(F.col('age'), F.col('Experience').desc()).show()


+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Lucy|NULL|      NULL| 40000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Paul|  24|         3| 20000|
|Sunny|  29|         4| 20000|
| Lucy|  30|         8| 25000|
| Paul|  31|        10| 30000|
| NULL|  34|        10| 38000|
| NULL|  36|      NULL|  NULL|
+-----+----+----------+------+



### 9. Join & Union

In [175]:
df2 = spark.read.csv('/content/test2.csv', header=True)
df2.show()

+-----+----+----------+------+
| Name| age|Experience|Salary|
+-----+----+----------+------+
| Paul|  31|        10| 30000|
| Lucy|  30|         8| 25000|
|Sunny|  29|         4| 20000|
| Paul|  24|         3| 20000|
|  Ann|  21|         1| 15000|
|  Tom|  23|         2| 18000|
| Lucy|NULL|      NULL| 40000|
| NULL|  34|        10| 38000|
| NULL|  36|      NULL|  NULL|
+-----+----+----------+------+



In [176]:
df3 = spark.read.csv('/content/test3.csv', header=True)
df3.show()

+-----+------------+------+
| Name| Departments|salary|
+-----+------------+------+
|  Ann|Data Science| 10000|
|  Ann|         IOT|  5000|
|  Tom|    Big Data|  4000|
|  Ann|    Big Data|  4000|
|  Tom|Data Science|  3000|
|Peter|Data Science| 20000|
|Peter|         IOT| 10000|
|Peter|    Big Data|  5000|
|Sunny|Data Science| 10000|
|Sunny|    Big Data|  2000|
+-----+------------+------+



In [ ]:
# 1) 이름이 다른 컬럼들 끼리의 Join

df2 = df2.withColumnRenamed('Name', 'new_Name')
df2.show()

In [ ]:
# df2.new_Name <----> df3.Name

df2.join(df3, df2.new_Name == df3.Name, 'outer').show() # inner:default, left, right, outer ...

In [ ]:
# 2) 이름이 동일한 컬럼들 끼리의 join

# df2.join(df3, df2.Name == df3.Name).show()

df2.join(df3, ['Name']).show()

In [165]:
# 3) 여러 컬럼들 끼리 join

df2.join(df3, (df2.Name == df3.Name) & (df2.Salary == df3.salary)).show()

+----+---+----------+------+----+-----------+------+
|Name|age|Experience|Salary|Name|Departments|salary|
+----+---+----------+------+----+-----------+------+
+----+---+----------+------+----+-----------+------+



In [166]:
df2 = df2.withColumnRenamed('Salary', 'salary')
df2.columns

['Name', 'age', 'Experience', 'salary']

In [167]:
# 4) 이름이 동일한 여러 컬럼들 끼리 join

df2.join(df3, ['Name', 'salary']).show()

+----+------+---+----------+-----------+
|Name|salary|age|Experience|Departments|
+----+------+---+----------+-----------+
+----+------+---+----------+-----------+



In [168]:
df1 = spark.read.csv('/content/test1.csv', header=True)
df2 = spark.read.csv('/content/test2.csv', header=True)

In [169]:
print(df1.columns)

print(df2.columns)

['Name', 'age', 'Experience', 'Salary']
['Name', 'age', 'Experience', 'Salary']


In [ ]:
# 1) Union (테이블들의 컬럼이 반드시 동일)

df1.union(df2).show()

In [ ]:
df1.union(df3).show()

In [174]:
# 2) UnionByName (테이블의 컬럼이 동일하지 않아도 됨)

df1.unionByName(df3, allowMissingColumns=True).show()

+-----+----+----------+------+------------+
| Name| age|Experience|Salary| Departments|
+-----+----+----------+------+------------+
| Paul|  31|        10| 10000|        NULL|
|  Tom|  30|         8| 25000|        NULL|
|Sunny|  29|         4| 40000|        NULL|
| Paul|  24|         3| 20000|        NULL|
|  Ann|  21|         1| 15000|        NULL|
|Peter|  23|         2| 18000|        NULL|
|  Ann|NULL|      NULL| 10000|Data Science|
|  Ann|NULL|      NULL|  5000|         IOT|
|  Tom|NULL|      NULL|  4000|    Big Data|
|  Ann|NULL|      NULL|  4000|    Big Data|
|  Tom|NULL|      NULL|  3000|Data Science|
|Peter|NULL|      NULL| 20000|Data Science|
|Peter|NULL|      NULL| 10000|         IOT|
|Peter|NULL|      NULL|  5000|    Big Data|
|Sunny|NULL|      NULL| 10000|Data Science|
|Sunny|NULL|      NULL|  2000|    Big Data|
+-----+----+----------+------+------------+



### 10. Window

> https://sparkbyexamples.com/pyspark/pyspark-window-functions/

In [183]:
df3 = df3.withColumn('salary', F.col('salary').cast(T.IntegerType()))

df3.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- salary: integer (nullable = true)



In [184]:
from pyspark.sql import window as W

# 1) window 변수 생성

window_var = W.Window.partitionBy('Name').orderBy('salary')

# 2) window 변수가 적용된 새로운 컬럼 생성

df3.withColumn('window', F.row_number().over(window_var)).show()

+-----+------------+------+------+
| Name| Departments|salary|window|
+-----+------------+------+------+
|  Ann|    Big Data|  4000|     1|
|  Ann|         IOT|  5000|     2|
|  Ann|Data Science| 10000|     3|
|Peter|    Big Data|  5000|     1|
|Peter|         IOT| 10000|     2|
|Peter|Data Science| 20000|     3|
|Sunny|    Big Data|  2000|     1|
|Sunny|Data Science| 10000|     2|
|  Tom|Data Science|  3000|     1|
|  Tom|    Big Data|  4000|     2|
+-----+------------+------+------+



### 11. UDF(User Defined Function)

> https://velog.io/@newnew_daddy/spark05

In [186]:
# 1) 직접 등록
def str_lower(var):
  return var.lower()

udf_lower = F.udf(str_lower, returnType = T.StringType())

df3.withColumn('lower_Departments', udf_lower(F.col('Departments'))).show()

'data science'

In [187]:
df3.columns

['Name', 'Departments', 'salary']

In [190]:
# 2) udf 데코레이터 사용

@F.udf(returnType = T.StringType())
def str_lower(var):
  return var.lower()

df3.withColumn('lower_Departments', str_lower(F.col('Departments'))).show()

+-----+------------+------+-----------------+
| Name| Departments|salary|lower_Departments|
+-----+------------+------+-----------------+
|  Ann|Data Science| 10000|     data science|
|  Ann|         IOT|  5000|              iot|
|  Tom|    Big Data|  4000|         big data|
|  Ann|    Big Data|  4000|         big data|
|  Tom|Data Science|  3000|     data science|
|Peter|Data Science| 20000|     data science|
|Peter|         IOT| 10000|              iot|
|Peter|    Big Data|  5000|         big data|
|Sunny|Data Science| 10000|     data science|
|Sunny|    Big Data|  2000|         big data|
+-----+------------+------+-----------------+



### 12. 기타
  - 

> https://velog.io/@newnew_daddy/spark01

> https://klefaloe4jbaezmvktwox2s3i40tudou.lambda-url.ap-northeast-2.on.aws/search

In [193]:
# 컬럼 이름 일괄 변환

change_cols = ['name', 'depart', '연봉']

df3.toDF(*change_cols).show(4)

+----+------------+-----+
|name|      depart| 연봉|
+----+------------+-----+
| Ann|Data Science|10000|
| Ann|         IOT| 5000|
| Tom|    Big Data| 4000|
| Ann|    Big Data| 4000|
+----+------------+-----+
only showing top 4 rows



In [194]:
# F.lit() -> 고정된 값을 넣어주는 함수

df3.withColumn('lit_col', F.lit('고정')).show()

+-----+------------+------+-------+
| Name| Departments|salary|lit_col|
+-----+------------+------+-------+
|  Ann|Data Science| 10000|   고정|
|  Ann|         IOT|  5000|   고정|
|  Tom|    Big Data|  4000|   고정|
|  Ann|    Big Data|  4000|   고정|
|  Tom|Data Science|  3000|   고정|
|Peter|Data Science| 20000|   고정|
|Peter|         IOT| 10000|   고정|
|Peter|    Big Data|  5000|   고정|
|Sunny|Data Science| 10000|   고정|
|Sunny|    Big Data|  2000|   고정|
+-----+------------+------+-------+



In [ ]:
# split() -> 특정 문자를 기준으로 문자를 split

df.withColumn('split', split(F.col("Departments"), ' ').getItem(0))

In [195]:
# pyspark DataFrame --> Pandas DataFrame  :  toPandas()
# Pandas DataFrame --> pyspark DataFrame  :  spark.createDataFrame()
pdf = df3.toPandas()

type(pdf)

pandas.core.frame.DataFrame

In [198]:
sdf = spark.createDataFrame(pdf)

type(sdf)

pyspark.sql.dataframe.DataFrame

In [199]:
pdf.to_csv('/content/test4.csv', index=False)